<a href="https://colab.research.google.com/github/Soham109/Big-5-personality-trait-detector/blob/main/Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim spacy nltk scikit-learn
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

splits = {
    'train': 'hf://datasets/Fatima0923/Automated-Personality-Prediction/Personality Datasets - Reddit/train_set.csv',
    'validation': 'hf://datasets/Fatima0923/Automated-Personality-Prediction/Personality Datasets - Reddit/val_set.csv',
    'test': 'hf://datasets/Fatima0923/Automated-Personality-Prediction/Personality Datasets - Reddit/eval_set.csv',
}

train_data = pd.read_csv(splits['train'])
val_data = pd.read_csv(splits['validation'])
test_data = pd.read_csv(splits['test'])

def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if token.is_alpha and token.text not in stop_words]
    return ' '.join(tokens)

train_data['processed_text'] = train_data['text'].apply(preprocess)
val_data['processed_text'] = val_data['text'].apply(preprocess)
test_data['processed_text'] = test_data['text'].apply(preprocess)

X_train = train_data['processed_text']
X_val = val_data['processed_text']
X_test = test_data['processed_text']

y_train = train_data[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']]
y_val = val_data[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']]
y_test = test_data[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

regressor = MultiOutputRegressor(RandomForestRegressor())
regressor.fit(X_train_tfidf, y_train)

y_pred = regressor.predict(X_val_tfidf)

from sklearn.metrics import mean_squared_error
for i, col in enumerate(y_train.columns):
    mse = mean_squared_error(y_val.iloc[:, i], y_pred[:, i])
    print(f"MSE for {col}: {mse}")




MSE for agreeableness: 445.59711383327004
MSE for openness: 334.6263373394649
MSE for conscientiousness: 302.16355090916517
MSE for extraversion: 458.8842164924528
MSE for neuroticism: 325.57824985932973


In [ ]:
import joblib
joblib.dump(regressor, 'personality_predictor.pkl')

['personality_predictor.pkl']

In [ ]:
import joblib
regressor2 = joblib.load('personality_predictor.pkl')